<a href="https://colab.research.google.com/github/yeray142/AIML_project/blob/master/notebooks/SVM_Basic_vesrion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4AI | SVM without "CODE" column & old vesrion

### Set up for Project

##### Library

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 3.9 MB/s 
     |████████████████████████████████| 209 kB 7.9 MB/s 
     |████████████████████████████████| 78 kB 2.8 MB/s 
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 147 kB 11.3 MB/s 
     |████████████████████████████████| 112 kB 16.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=625b06557a456a99a28b4dabcc94a52697b44679d654eedb9d22b8d341902c7e
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [3]:
# Modeling
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, KFold,StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.svm import SVC
from sklearn.metrics import log_loss

In [4]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

#### Data Load

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/ML4AI/cleaned_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/ML4AI/cleaned_test.csv")

In [7]:
df_train = df_train.drop(['Unnamed: 0','CODE'], axis=1)
df_test = df_test.drop(['index','CODE'],axis =1)

## Data EDA

#### **Basic EDA**

In [8]:
df_train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,No job,2.0,6,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,5,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,22,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,37,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,26,2.0


In [9]:
df_test.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,0,0,1,0,No job,2.0,60
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,8671,0,1,0,Core staff,2.0,36
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,217,1,1,0,Laborers,2.0,40
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,19270,2531,1,0,0,Drivers,2.0,41
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,17822,9385,1,0,0,Managers,2.0,8


In [10]:
df_train.describe()

,child_num,income_total,days_birth,days_employed,work_phone,home_phone,email,family_size,begin_month,credit
count,24817.000000,2.481700e+04,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000,24817.000000
mean,0.430874,1.868744e+05,15951.287625,2198.272112,0.225249,0.294274,0.091510,2.199057,26.054318,1.514325
std,0.750609,1.008656e+05,4205.102462,2371.033103,0.417754,0.455725,0.288339,0.919132,16.532853,0.704571
min,0.000000,2.700000e+04,7705.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,1.215000e+05,12443.000000,407.000000,0.000000,0.000000,0.000000,2.000000,12.000000,1.000000
50%,0.000000,1.575000e+05,15536.000000,1539.000000,0.000000,0.000000,0.000000,2.000000,24.000000,2.000000
75%,1.000000,2.250000e+05,19428.000000,3153.000000,0.000000,1.000000,0.000000,3.000000,39.000000,2.000000
max,19.000000,1.575000e+06,25152.000000,15713.000000,1.000000,1.000000,1.000000,20.000000,60.000000,2.000000


## Data modeling

### Data Preprocessing for construct model

1) Encoding of categorical features

In [11]:
num_col = df_train.dtypes[df_train.dtypes != "object"].index.tolist()
num_col.remove('credit')

cat_col = df_train.dtypes[df_train.dtypes == "object"].index.tolist()

In [12]:
num_col

['child_num',
 'income_total',
 'days_birth',
 'days_employed',
 'work_phone',
 'home_phone',
 'email',
 'family_size',
 'begin_month']

In [13]:
cat_col

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occup_type']

In [14]:
# StandardScaler
scaler = StandardScaler()
df_train[num_col] = scaler.fit_transform(df_train[num_col])
df_test[num_col] = scaler.fit_transform(df_test[num_col])

In [15]:
# OrdinalEncoding
Encoder = OrdinalEncoder()
df_train[cat_col] = Encoder.fit_transform(df_train[cat_col], df_train['credit'])
df_test[cat_col] = Encoder.fit_transform(df_test[cat_col])


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24817 entries, 0 to 24816
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         24817 non-null  float64
 1   car            24817 non-null  float64
 2   reality        24817 non-null  float64
 3   child_num      24817 non-null  float64
 4   income_total   24817 non-null  float64
 5   income_type    24817 non-null  float64
 6   edu_type       24817 non-null  float64
 7   family_type    24817 non-null  float64
 8   house_type     24817 non-null  float64
 9   days_birth     24817 non-null  float64
 10  days_employed  24817 non-null  float64
 11  work_phone     24817 non-null  float64
 12  home_phone     24817 non-null  float64
 13  email          24817 non-null  float64
 14  occup_type     24817 non-null  float64
 15  family_size    24817 non-null  float64
 16  begin_month    24817 non-null  float64
 17  credit         24817 non-null  float64
dtypes: flo

3) Data Split ⇒ We need to split with stratified split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:,df_train.columns != 'credit'], df_train['credit'], test_size = 0.2)

In [18]:
X_train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,days_birth,days_employed,work_phone,home_phone,email,occup_type,family_size,begin_month
5798,0.0,1.0,1.0,-0.574044,0.377992,4.0,1.0,1.0,1.0,0.344996,0.402671,-0.539200,1.54861,-0.317376,3.0,-0.216575,-0.245233
17563,0.0,0.0,0.0,2.090514,-0.871220,2.0,4.0,1.0,1.0,-0.378426,1.038272,-0.539200,1.54861,3.150839,1.0,1.959434,0.722557
14527,1.0,1.0,1.0,-0.574044,-0.960450,4.0,4.0,0.0,1.0,-0.744417,-0.861782,-0.539200,-0.64574,-0.317376,8.0,-0.216575,-0.910588
19995,0.0,0.0,0.0,-0.574044,-1.138909,4.0,4.0,3.0,1.0,0.677218,-0.308847,1.854598,-0.64574,-0.317376,1.0,-1.304580,-0.063772
17773,0.0,1.0,1.0,-0.574044,-0.960450,2.0,4.0,1.0,1.0,0.792556,0.171122,1.854598,1.54861,-0.317376,3.0,-0.216575,0.662070


In [19]:
y_train

5798     1.0
17563    2.0
14527    2.0
19995    2.0
17773    2.0
        ... 
4369     2.0
2624     2.0
18203    0.0
16757    1.0
1014     2.0
Name: credit, Length: 19853, dtype: float64

### Modeling

- Optuna

In [20]:
def objective(trial: Trial) -> float:
    params_svc = {
        "random_state": 42,
        'kernel': 'rbf',
        'C' : trial.suggest_loguniform('C', 1e-5,1e+5),
        'gamma' : trial.suggest_loguniform('gamma', 1e-5,1e+5),
        'probability' : True,
    }
    model = sklearn.svm.SVC(**params_svc)
    model.fit(
        X_train,
        y_train
    )

    svc_pred = model.predict_proba(X_val)
    log_score = log_loss(y_val, svc_pred)
    
    return log_score

In [21]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="svc_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2022-12-11 10:05:50,330] A new study created in memory with name: svc_parameter_opt
[I 2022-12-11 10:13:06,495] Trial 0 finished with value: 0.8644425539006468 and parameters: {'C': 0.05564180225431373, 'gamma': 32147.193482816965}. Best is trial 0 with value: 0.8644425539006468.
[I 2022-12-11 10:26:26,232] Trial 1 finished with value: 0.8797192548974977 and parameters: {'C': 208.90047049266641, 'gamma': 9.695826644515218}. Best is trial 0 with value: 0.8644425539006468.
[I 2022-12-11 10:28:22,044] Trial 2 finished with value: 0.9592704268115463 and parameters: {'C': 0.0003632339256943143, 'gamma': 0.0003630322466779861}. Best is trial 0 with value: 0.8644425539006468.
[I 2022-12-11 10:30:03,950] Trial 3 finished with value: 0.8763995789423636 and parameters: {'C': 3.809220577048033e-05, 'gamma': 4589.458612326462}. Best is trial 0 with value: 0.8644425539006468.
[I 2022-12-11 10:42:55,610] Trial 4 finished with value: 0.8898465595391098 and parameters: {'C': 10.260065124896771, 'ga

Best Score: 0.8410280841565297
Best trial: {'C': 0.011026112761510002, 'gamma': 1.7683340776662542}


In [22]:
study.best_params['C']

0.011026112761510002

In [23]:
study.best_params['gamma']

1.7683340776662542

--- 
K Fold

In [24]:
p_optuna = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : study.best_params['C'],
    'gamma' : study.best_params['gamma'],
    'probability' : True,
}

In [25]:
def stratified_kfold_svc(p, n_fold, X, y, X_test):
    # Declaring Stratified K-Fold:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    
    # Initializing arrays:
    svc_oof = np.zeros((X.shape[0], 3))
    svc_preds = np.zeros((X_test.shape[0], 3))

    # Main loop:
    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # Model fit:
        model = SVC(**p)
        model.fit(X_train, y_train)

        svc_oof[valid_idx] = model.predict_proba(X_valid) # Infers the class probability of train dataset
        svc_preds += model.predict_proba(X_test) / n_fold # Infers the class probability of test dataset
        print(f"Log Loss Score: {log_loss(y_valid, svc_oof[valid_idx]):.5f}")

    log_score = log_loss(y, svc_oof)
    print(f"\nLog Loss Score: {log_score:.5f}")

    return svc_oof, svc_preds

In [26]:
svc_oof_optuna, svc_preds_optuna = stratified_kfold_svc(p_optuna, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.83010
============ Fold 1 ============
Log Loss Score: 0.83507
============ Fold 2 ============
Log Loss Score: 0.83305
============ Fold 3 ============
Log Loss Score: 0.84527
============ Fold 4 ============
Log Loss Score: 0.84105
============ Fold 5 ============
Log Loss Score: 0.82761
============ Fold 6 ============
Log Loss Score: 0.83344
============ Fold 7 ============
Log Loss Score: 0.83781
============ Fold 8 ============
Log Loss Score: 0.83072
============ Fold 9 ============
Log Loss Score: 0.84204

Log Loss Score: 0.83562


In [27]:
p_grid = {
    'kernel' : 'rbf',
    "random_state": 42,
    'C' : 10.0,
    'gamma' : 0.1,
    'probability' : True,
}

In [28]:
svc_oof_grid, svc_preds_grid = stratified_kfold_svc(p_grid, 10, df_train.drop("credit", axis=1),df_train['credit'] , df_test)

============ Fold 0 ============
Log Loss Score: 0.80175
============ Fold 1 ============
Log Loss Score: 0.81615
============ Fold 2 ============
Log Loss Score: 0.80739
============ Fold 3 ============
Log Loss Score: 0.80799
============ Fold 4 ============
Log Loss Score: 0.81509
============ Fold 5 ============
Log Loss Score: 0.79904
============ Fold 6 ============
Log Loss Score: 0.81455
============ Fold 7 ============
Log Loss Score: 0.80565
============ Fold 8 ============
Log Loss Score: 0.80913
============ Fold 9 ============
Log Loss Score: 0.81197

Log Loss Score: 0.80887


In [29]:
df_submission = pd.read_csv('/content/drive/MyDrive/ML4AI/sample_submission.csv')

In [30]:
df_submission.set_index("index", inplace=True)
df_submission[:] = svc_preds_grid
df_submission.head(20)

,0,1,2
index,,,
26457,0.116412,0.165037,0.718551
26458,0.303468,0.216466,0.480066
26459,0.094742,0.170666,0.734592
26460,0.079404,0.177906,0.742690
26461,0.102681,0.373569,0.523749
26462,0.074945,0.206909,0.718146
26463,0.287796,0.122792,0.589412
26464,0.115077,0.201079,0.683843
26465,0.106574,0.082342,0.811084


In [31]:
df_submission.to_csv('/content/drive/MyDrive/ML4AI/SVC_Basic.csv',index=False)